In [ ]:
import os
import sys
import time as t_util
import numpy as np
import pandas as pd
import yaml
import cftime
import xarray as xr
import matplotlib.pyplot as plt
import mplotutils as mpu
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy import stats


## Define folders

In [ ]:
#Read main path
with open('../path_main.txt', 'r') as file:    path_main  = file.read()

dir_scripts = f'{path_main}Scripts/'
dir_names   = f'{path_main}Scripts/Model_lists/'
dir_CORDEX  = f'{path_main}Data/EURO-CORDEX/JJA/'
dir_EOBS    = f'{path_main}Data/EOBS/JJA/'
dir_ERA     = f'{path_main}Data/ERA5-Land/JJA/'
dir_STA     = f'{path_main}Data/Stations/JJA/'
dir_GSOD    = f'{path_main}Data/GSOD/JJA/'
dir_fig     = f'{path_main}Figures/Paper_v2/'
if not os.path.exists(dir_fig): os.mkdir(dir_fig)
    

## Prepare variables and parameters

In [ ]:
#Define cities
cities = ['Lisbon', 'Madrid', 'Barcelona', 'Rome', 'Athens', 'Istanbul', 'Sofia', 'Bucharest', 'Belgrade', 'Zagreb',
          'Milan', 'Budapest', 'Munich', 'Vienna', 'Prague', 'Paris', 'Brussels', 'Amsterdam', 'London', 'Dublin',
          'Hamburg', 'Copenhagen', 'Berlin', 'Warsaw', 'Kharkiv', 'Kyiv', 'Minsk', 'Vilnius', 'Riga', 'Moscow',
          'NizhnyNovgorod', 'Kazan', 'SaintPetersburg', 'Helsinki', 'Stockholm', 'Oslo']

#Define scenarios and variables
vars_ERA5 = ['mintemp2m', 'maxtemp2m']
vars_STAT = ['TN', 'TX']
vars_EOBS = ['tn', 'tx']
var_names = ['tasmin', 'tasmax']

# Load city coordinates
fname_coords = dir_scripts + 'City_coordinates.yml'
with open(fname_coords, 'r') as file:
    city_coords = yaml.safe_load(file)

#Define scenarios and variables
RCP = 'rcp85'

#Define models and RCPs which should be used
all_models = dict()
all_models = []
with open(dir_names + 'Models_CORDEX-EUR-11_RCP85.txt', 'r') as filehandle:
    for line in filehandle:
        all_models.append(eval(line[:-1]))


In [ ]:
def get_city_data(data, city, city_coords, var_name, data_source):
    
    #Convert longitude from [0, 360] to [-180, 180]
    if 'longitude' in data.coords:  lat_name, lon_name = 'latitude', 'longitude'
    elif 'lon' in data.coords:      lat_name, lon_name = 'lat', 'lon'
    if data[lon_name].max()>180:
        data[lon_name] = data[lon_name].where(data[lon_name]<180, ((data[lon_name] + 180) % 360) - 180)

    #Get lat and lon of city
    lat_sel, lon_sel = city_coords[city]
    
    #Find grid point closest to city
    if data_source in ['EOBS', 'ERA5-Land']:
        
        #Find grid point closest to city
        lat_city = np.argmin(np.abs(data[lat_name].values - lat_sel))
        lon_city = np.argmin(np.abs(data[lon_name].values - lon_sel))

        #Select NxN box around grid point
        N = 11
        N_half = int((N-1)/2)
        lat_rng  = slice(lat_city - N_half, lat_city + N_half + 1)
        lon_rng  = slice(lon_city - N_half, lon_city + N_half + 1)
        data_sel = data.isel(latitude=lat_rng, longitude=lon_rng)        
        
    elif data_source=='EURO-CORDEX':

        #Find grid point closest to city
        loc_city = (np.abs(data[lon_name] - lon_sel)) + (np.abs(data[lat_name] - lat_sel))
        ind_city = np.unravel_index(np.argmin(loc_city.values), loc_city.shape)

        #Select NxN box around grid point
        N = 9
        N1 = int(N/2 - 0.5)
        N2 = int(N/2 + 0.5)
        lat_rng  = slice(ind_city[0] - N1, ind_city[0] + N2)
        lon_rng  = slice(ind_city[1] - N1, ind_city[1] + N2)
        
    else:
        
        sys.exit('Data source not defined')

    if 'rlat' in data.dims:   data_city = data.isel(rlat=lat_rng, rlon=lon_rng)
    elif 'x' in data.dims:    data_city = data.isel(y=lat_rng, x=lon_rng)
    else:                     data_city = data.isel(latitude=lat_rng, longitude=lon_rng)

    #Calculate distance from city center
    dist = np.sqrt((data_city[lat_name] - lat_sel)**2 + (data_city[lon_name] - lon_sel)**2)                

    #Convert K to °C
    if data_city[var_name].mean()>200:
        data_city = data_city - 273.15    

    return(data_city, dist)


## Read data

In [ ]:
data_CORD = dict()
data_EOBS = dict()
data_ERA5 = dict()
data_STAT = dict()
data_GSOD = dict()


## EURO-CORDEX ##
        
data_source = 'EURO-CORDEX'
print("Preparing " + data_source)
    
#Loop over variables
for variab in var_names:

    #Loop over models
    for i1, city in enumerate(cities):

        #Get file name
        dir_variab = dir_CORDEX + variab + '/'
        
        for model in all_models:
            
            #Get file name
            fnames = [file for file in os.listdir(dir_variab) if variab in file and model[0] in file and model[1] in file and model[2] in file and RCP in file]
            fnames = [file for file in fnames if variab + '_JJA-mean' in file and '1981-2010' in file]
            fnames = [file for file in fnames if 'masked-sea' in file]
            
            if len(fnames)!=1:
                sys.exit('Filename not unique')                

            #Read data
            data = xr.open_dataset(dir_variab + fnames[0])
            
            if (variab in ['tasmin', 'tasmax']) and (data[variab].mean().values.item()<200):
                print(model)
                print(data[variab].mean().values.item())

            #Get data around city
            data_city, dist = get_city_data(data, city, city_coords, variab, data_source)
            
            if data_city[variab].mean().values.item()<0:
                print(model)
                print(variab)
                print(data_city[variab].mean().values.item())            
            
            #Save in dict
            data_CORD['data_' + city + '_' + "_".join(model) + '_' + variab] = data_city
            data_CORD['dist_' + city + '_' + "_".join(model) + '_' + variab] = dist

            
## ERA5-Land ##

data_source = 'ERA5-Land'
print("Preparing " + data_source)

#Loop over variables
for variab, var_out in zip(vars_ERA5, var_names):
    
    #Loop over models
    for i1, city in enumerate(cities):

        #Read data
        fname = dir_ERA + variab + '/' + variab + '_JJA-mean_ERA5-Land_day_1981-2010.nc'
        data = xr.open_dataset(fname)

        #Get data around city
        data_city, dist = get_city_data(data, city, city_coords, variab, data_source)
        
        #Rename
        data_city = data_city.rename({variab: var_out})
        
        #Save in dict
        data_ERA5['data_' + city + '_' + var_out] = data_city
        data_ERA5['dist_' + city + '_' + var_out] = dist

        
## EOBS ##

data_source = 'EOBS'
print("Preparing " + data_source)


#Loop over variables
for variab, var_out in zip(vars_EOBS, var_names):

    #Loop over models
    for i1, city in enumerate(cities):

        #Read data
        fname = dir_EOBS + variab + '/' + variab + '_JJA-mean_EOBS_day_1981-2010.nc'
        data = xr.open_dataset(fname)

        #Get data around city
        data_city, dist = get_city_data(data, city, city_coords, variab, data_source)
        
        #Rename
        data_city = data_city.rename({variab: var_out})
        
        #Save in dict
        data_EOBS['data_' + city + '_' + var_out] = data_city
        data_EOBS['dist_' + city + '_' + var_out] = dist
        
        
## STATIONS ##

print("Preparing STATIONS")

#Loop over variables
for var_name, var_out in zip(vars_STAT, var_names):

    #Loop over models
    for i1, city in enumerate(cities):

        #File name
        fname_STA = dir_STA + var_name + '_Stations_' + city + ".csv"
        fname_GSOD = dir_GSOD + var_name + '_GSOD-stations_' + city + ".csv"
        
        #Get lat and lon of city
        lat_sel, lon_sel = city_coords[city]        
        
        #Check if station data exist
        if os.path.exists(fname_STA):
            
            #Read station data
            data_STA = pd.read_csv(fname_STA)
            
            #Get distance to city center
            latS, lonS = data_STA.iloc[:,0], data_STA.iloc[:,1]
            distS = np.sqrt((latS - lat_sel)**2 + (lonS - lon_sel)**2)   

            #Rename
            data_STA = data_STA.rename({var_name: var_out}, axis='columns')
            
        else:
            
            print('No station data for ' + city)
            data_STA = []
            distS = []
            
        #Check if station data exist
        if os.path.exists(fname_GSOD):
            
            #Read station data
            data_GS = pd.read_csv(fname_GSOD)

            #Get distance to city center
            latG, lonG = data_GS.iloc[:,0], data_GS.iloc[:,1]
            distG = np.sqrt((latG - lat_sel)**2 + (lonG - lon_sel)**2)   

            #Rename
            data_GS = data_GS.rename({var_name: var_out}, axis='columns')
            
        else:
            
            print('No station data for ' + city)
            data_STA = []
            distS = []
                
        
                
        #Save in dict
        data_STAT['data_' + city + '_' + var_out] = data_STA
        data_STAT['dist_' + city + '_' + var_out] = distS
        
        
        
## GSOD stations ##
            
print("Preparing GSOD")
    
#Loop over variables
for var_name, var_out in zip(vars_STAT, var_names):

    #Loop over models
    for i1, city in enumerate(cities):

        #File name
        fname_GSOD = dir_GSOD + var_name + '_GSOD-stations_' + city + ".csv"
        
        #Check if GSOD station data exist
        if os.path.exists(fname_GSOD):
            
            #Read GSOD station data
            data_GS = pd.read_csv(fname_GSOD)

            #Get lat and lon of city
            lat_sel, lon_sel = city_coords[city]

            #Get distance to city center
            latS, lonS = data_GS.iloc[:,0], data_GS.iloc[:,1]
            distS = np.sqrt((latS - lat_sel)**2 + (lonS - lon_sel)**2)   

            #Rename
            data_GS = data_GS.rename({var_name: var_out}, axis='columns')
            
        else:
            
            print('No GSOD station data for ' + city)
            data_GS = []
            distS = []
        
        #Save in dict
        data_GSOD['data_' + city + '_' + var_out] = data_GS
        data_GSOD['dist_' + city + '_' + var_out] = distS


## Plot T as function of city center distance

In [ ]:
vars_plot   = ['tasmax', 'tasmin']#, 'huss']
vars_legend = ['TX', 'TN']#, 'q']

#Define method for interpolation
met_scatter = 'quantiles-filling'
# met_scatter = 'gaussian-kde'

#Loop over variables
for variab, var_leg in zip(vars_plot, vars_legend):
    
    #Create figures
    fig, axes = plt.subplots(6, 6, figsize=(14, 10))     
    axes = axes.flatten()
    plt.subplots_adjust(hspace=0.1, wspace=0.2)

    #Loop over models
    for i1, city in enumerate(sorted(cities)):

        #Get axes
        ax = axes[i1]

        
        #### EURO-CORDEX ####
        
        CORD_x = []
        CORD_y = []
        for model in all_models:
            
            #Get data
            x = data_CORD['dist_' + city + '_' + "_".join(model) + '_' + variab].values.flatten()
            y = data_CORD['data_' + city + '_' + "_".join(model) + '_' + variab][variab].values.flatten()
            
            #Collect data to define limits
            CORD_x.append(x)
            CORD_y.append(y)
    
        x  = np.array(CORD_x).flatten()
        y  = np.array(CORD_y).flatten()
        x = x[~np.isnan(y)]
        y = y[~np.isnan(y)]
        
        if met_scatter=='gaussian-kde':

            xy = np.vstack([x,y])

            x1 = np.linspace(np.min(x), np.max(x), 100)
            y1 = np.linspace(np.min(y), np.max(y), 100)
            xy_mesh = np.array(np.meshgrid(x1, y1))
            x1 = xy_mesh[0,:,:].flatten()
            y1 = xy_mesh[1,:,:].flatten()

            # Calculate the point density
            z1 = stats.gaussian_kde(xy)([x1, y1])


            # Sort the points by density, so that the densest points are plotted last
            idx = z1.argsort()
            x1, y1, z1 = x1[idx], y1[idx], z1[idx]

            ax.scatter(x1, y1, c=z1, s=4**2, cmap='Greys')
            
        elif met_scatter=='quantiles-filling':

            alphas = [0.03, 0.15, 0.3, 0.15, 0.03]

            
            bins      = np.linspace(x.min(), x.max(), 10)
            quantiles = [0.01, 0.1, 0.25, 0.75, 0.9, 0.99, 0.5]
            # quantiles = [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1]

            y_q = np.empty((len(bins)-1, len(quantiles))) * np.NaN
            x_q = []
            for iq1, (b1, b2) in enumerate(zip(bins[0:-1], bins[1::])):

                y_sel = y[(x>=b1) & (x<b2)]
                x_q.append(np.mean([b1, b2]))

                for iq2, q in enumerate(quantiles):

                    y_q[iq1, iq2] = np.quantile(y_sel, q)

            
            x_q = np.append(np.min(x), x_q)
            y0 = y_q[0,:]
            y0 = np.transpose(np.expand_dims(y0, 1))
            y_q = np.concatenate((y0, y_q), axis=0)
                    
            for i2, (q, alpha) in enumerate(zip(quantiles[0:-2], alphas)):
                ax.fill_between(x_q, y_q[:,i2], y_q[:,i2+1], color='k', alpha=alpha, edgecolor='none')

            p_CORD = ax.plot(x_q, y_q[:,-1], 'k') 
            
    


        #### ERA5 ####
            
        #Get data
        ERA5_plot_x = data_ERA5['dist_' + city + '_' + variab]
        ERA5_plot_y = data_ERA5['data_' + city + '_' + variab]

        #Plot ERA5
        ax.scatter(ERA5_plot_x, ERA5_plot_y[variab], marker='o', s=4**2, color='lightgray',
                   edgecolor='darkred', linewidth=0.5, alpha=1, zorder=10)
                


        #### EOBS ####

        if variab in ['tasmin', 'tasmax']:
            
            #Get data
            EOBS_plot_x = data_EOBS['dist_' + city + '_' + variab]
            EOBS_plot_y = data_EOBS['data_' + city + '_' + variab]

            #Plot EOBS
            ax.scatter(EOBS_plot_x, EOBS_plot_y[variab], marker='o', s=4**2, color='lightgray',
                       edgecolor='tab:blue', linewidth=0.5, alpha=1, zorder=15)


        #### STATIONS ####
            
        if variab in ['tasmin', 'tasmax']:
            
            #Get data
            STAT_plot_x = data_STAT['dist_' + city + '_' + variab]
            STAT_plot_y = data_STAT['data_' + city + '_' + variab]

            #Plot stations
            if type(STAT_plot_y) is not list:
                colors = np.empty(len(STAT_plot_x), dtype='object')
                colors[STAT_plot_y['flag']==1] = 'tab:blue'
                colors[STAT_plot_y['flag']==2] = 'tab:blue'#'lightblue'
                ax.scatter(STAT_plot_x, STAT_plot_y[variab], marker='o', s=4**2, color=colors, 
                           edgecolor='tab:blue', linewidth=0.5, zorder=20)
            
            
        #### GSOD STATIONS ####
            
        if variab in ['tasmin', 'tasmax']:
            
            #Get data
            GSOD_plot_x = data_GSOD['dist_' + city + '_' + variab]
            GSOD_plot_y = data_GSOD['data_' + city + '_' + variab]

            #Plot stations
            if type(GSOD_plot_y) is not list:
                colors = np.empty(len(GSOD_plot_x), dtype='object')
                colors[GSOD_plot_y['flag']==1] = 'tab:blue'
                colors[GSOD_plot_y['flag']==2] = 'tab:blue'#'lightblue'
                ax.scatter(GSOD_plot_x, GSOD_plot_y[variab], marker='o', s=4**2, color=colors, 
                           edgecolor='tab:blue', linewidth=0.5, zorder=20)
            
        #x-ticks and labels
        if i1<30:
            ax.set_xticklabels([])
        else:
            ax.set_xlabel('Center distance [°]')
        if np.mod(i1,6)==0:
            ax.set_ylabel(var_leg + ' [°C]')
            
        #Set limits
        ylim1 = np.quantile(y, 0.01)
        ylim2 = np.quantile(y, 0.99)
        ylim_d = ylim2 - ylim1
        ylim1 = ylim1 - 0.1 * ylim_d
        ylim2 = ylim2 + 0.1 * ylim_d

        lims_ERA5 = ERA5_plot_y.where(ERA5_plot_x<0.51)
        lims_ERA5 = lims_ERA5[variab].values.flatten()
        if variab in ['tasmin', 'tasmax']:
            lims_EOBS = EOBS_plot_y.where(EOBS_plot_x<0.51)
            lims_EOBS = lims_EOBS[variab].values.flatten()
            if len(GSOD_plot_y)!=0:  lims_GSOD = np.array(GSOD_plot_y[(GSOD_plot_x<0.51).values][variab])
            else:                    lims_GSOD = [np.NaN]
            if len(STAT_plot_y)!=0:  lims_STAT = np.array(STAT_plot_y[(STAT_plot_x<0.51).values][variab])
            else:                    lims_STAT = [np.NaN]
            lims_conc = np.concatenate((lims_ERA5, lims_EOBS, lims_GSOD, lims_STAT))
        else:
            lims_conc = lims_ERA5
            
        #Set limits
        ylim1 = np.min([np.nanmin(lims_conc) - 0.05*(ylim2 - ylim1), ylim1])
        ylim2 = np.max([np.nanmax(lims_conc) + 0.05*(ylim2 - ylim1), ylim2])
        ax.set_ylim([ylim1, ylim2])
        ax.set_xlim([-0.01, 0.5])
    
        if variab!='huss':
            yl = np.diff(ax.get_ylim())
            if yl<6:
                yticks = np.arange(np.ceil(ylim1), np.ceil(ylim1) + 7, 2)
            elif (yl>=6) & (yl<9):
                yticks = np.arange(np.ceil(ylim1)+1, np.ceil(ylim1) + 10, 2)

            elif (yl>=9) & (yl<12):
                yticks = np.arange(np.ceil(ylim1), np.ceil(ylim1) + 12, 3)
            elif (yl>=12) & (yl<16):
                yticks = np.arange(np.ceil(ylim1), np.ceil(ylim1) + 16, 4)

            else:
                yticks = np.arange(np.ceil(ylim1), np.ceil(ylim1) + yl + 5, 5)

            ax.set_yticks(yticks)
            
        #Set limits (to be sure)
        ax.set_ylim([ylim1, ylim2])

        #Write city names
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        x_txt = xlim[0] + 0.50 * np.diff(xlim)
        y_txt = ylim[1] - 0.03 * np.diff(ylim)

        if city=='NizhnyNovgorod':     city_out = 'Nizhny Novgorod'
        elif city=='SaintPetersburg':  city_out = 'Saint Petersburg'
        else:                          city_out = city
                
        #Write city name
        ax.text(x_txt, y_txt, city_out, fontsize=8, ha='center', va='top', zorder=50, bbox=dict(facecolor='white', edgecolor='none', pad=0.5, alpha=0.5))
        
        #Define direction of ticks
        ax.tick_params(direction='in')
        
        #Put spines in front
        for k, spine in ax.spines.items():
            spine.set_zorder(30)
        
    #Create helper plots for legend
    p_help1 = ax.scatter(4, 15, marker='o', s=7**2, color='lightgray', edgecolor='darkred')
    p_help2 = ax.scatter(4, 15, marker='o', s=7**2, color='lightgray', edgecolor='tab:blue')
    p_help3 = ax.scatter(4, 15, marker='o', s=7**2, color='tab:blue', edgecolor='tab:blue')
    
    #Save figure
    if variab=='tasmax':
        fig_name = 'Fig3_'
    elif variab=='tasmin':
        fig_name = 'FigS2_'
    else:
        fig_name = 'FigS_'
        
    #Legend
    ax.legend([p_CORD[0], p_help1, p_help2, p_help3], ['EURO-CORDEX', 'ERA5-Land', 'E-OBS', 'Station data'],
              frameon=False, ncol=5, fontsize=12, loc=9, bbox_to_anchor=(-2.5, -0.5), scatteryoffsets=[0.6])
        
    #Save figure
    fig.savefig(dir_fig + fig_name + 'Validation_EURO-CORDEX_' + variab + '-JJA_1981-2010.png', dpi=200, bbox_inches='tight')
    

## Plot distribution

In [ ]:
#Select whether to include EOBS or not
EOBS_incl = 1

#Select variables to plot
vars_plot  = ['tasmax']#'tasmin', 
vars_legend = ['TX']#'TN', 

#Loop over variables
for variab, var_leg in zip(vars_plot, vars_legend):
    
    #Create figures
    fig, axes = plt.subplots(6, 6, figsize=(14, 10))     
    axes = axes.flatten()
    plt.subplots_adjust(hspace=0.3, wspace=0.1)

    #Loop over models
    for i1, city in enumerate(sorted(cities)):

        #Get axes
        ax = axes[i1]

        
        #### EURO-CORDEX ####
        
        #EURO-CORDEX
        CORD_all = np.empty(0)
        for model in all_models:
            
            #Get data
            CORD_dist = data_CORD['dist_' + city + '_' + "_".join(model) + '_' + variab]
            CORD_vals = data_CORD['data_' + city + '_' + "_".join(model) + '_' + variab]
            
            #Get data in 0.5° radius 
            CORD_coll = CORD_vals.where(CORD_dist<0.5)['tasmax'].values.flatten()
            CORD_coll = CORD_coll[~np.isnan(CORD_coll)]
            
            #Collect data in one array
            CORD_all = np.concatenate((CORD_all, CORD_coll))
            
            #Plot distribution
            p1 = pd.DataFrame(CORD_coll).plot.kde(ax=ax, color='k', alpha=0.1)        

        #Collect median
        CORD_med = np.median(CORD_all)
        
#         #Get y-values of median
#         CORD_med_y = np.empty(0)
#         for line in p1.get_lines():
#             x1, y1 = line.get_data()
#             CORD_med_line = y1[np.argmin(np.abs(x1 - CORD_med))]
#             CORD_med_y = np.concatenate((CORD_med_y, [CORD_med_line]))
            
        
        #### ERA5 ####
    
        #Get data
        ERA5_dist = data_ERA5['dist_' + city + '_' + variab]
        ERA5_vals = data_ERA5['data_' + city + '_' + variab]

        #Get data in 0.5° radius 
        ERA5_coll = ERA5_vals.where(ERA5_dist<0.5)['tasmax'].values.flatten()
        ERA5_coll = ERA5_coll[~np.isnan(ERA5_coll)]

        #Collect median
        ERA5_med = np.median(ERA5_coll)
        
        #Plot distribution
        p2 = pd.DataFrame(ERA5_coll).plot.kde(ax=ax, linestyle='-', color='darkred')
        
#         #Get y-values of median
#         x2, y2 = p2.get_lines()[-1].get_data()
#         ERA5_med_y = y2[np.argmin(np.abs(x2 - ERA5_med))]
        
    
        #### EOBS ####

        if EOBS_incl==1:
            
            EOBS_dist = data_EOBS['dist_' + city + '_' + variab]
            EOBS_vals = data_EOBS['data_' + city + '_' + variab]

            EOBS_coll = EOBS_vals.where(EOBS_dist<0.5)['tasmax'].values.flatten()
            EOBS_coll = EOBS_coll[~np.isnan(EOBS_coll)]

            #Collect median
            EOBS_med = np.median(EOBS_coll)

            #Plot distribution
            if len(EOBS_coll)!=0:
                p2 = pd.DataFrame(EOBS_coll).plot.kde(ax=ax, linestyle='-', color='tab:blue')

#             #Get y-values of median
#             x2, y2 = p2.get_lines()[0].get_data()
#             EOBS_med_y = y2[np.argmin(np.abs(x2 - EOBS_med))]

        #Plot medians
        ylims = ax.get_ylim()
        med_y = ylims[0] + 0.1 * np.diff(ylims)
        ax.plot(CORD_med, 0, marker='o', markersize=5, linestyle='none', color='k')
        ax.plot(ERA5_med, 0, marker='o', markersize=5, linestyle='none', color='darkred')
        if EOBS_incl==1:
            ax.plot(EOBS_med, 0, marker='o', markersize=5, linestyle='none', color='tab:blue')
            
        #x-ticks and labels
        ax.set_yticklabels([])
        if i1>=30:
            ax.set_xlabel('TX / °C')
        if np.mod(i1,6)==0:
            ax.set_ylabel('Density')
        else:
            ax.set_ylabel('')
            
        #Set limits
        xlim1 = np.quantile(CORD_all, 0.01)
        xlim2 = np.quantile(CORD_all, 0.99)
        xlim_d = xlim2 - xlim1
        xlim1 = xlim1 - 0.1 * xlim_d
        xlim2 = xlim2 + 0.1 * xlim_d
        #ax.set_ylim([ylim1, ylim2])
        ax.set_xlim([xlim1, xlim2])
        
        #Set x-ticks
        ax.set_xticks(np.arange(np.ceil(xlim1), xlim2, 2))        
        
        if city=='NizhnyNovgorod':     city_out = 'Nizhny Novgorod'
        elif city=='SaintPetersburg':  city_out = 'Saint Petersburg'
        else:                          city_out = city        
        
        #Write cities
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        x_txt = xlim[0] + 0.05 * np.diff(xlim)
        y_txt = ylim[1] - 0.05 * np.diff(ylim)
        ax.text(x_txt, y_txt, city_out, fontsize=8, ha='left', va='top') 
        
        ax.tick_params(direction='in')
        
        ax.get_legend().remove()
    
    #Helper plot for legend
    p_help = ax.plot([-200, -180], [0, 0], color='k', alpha=0.5)
    ax.set_xlim([xlim1, xlim2])
    
    #Prepare legend and output string
    if EOBS_incl==1: 
        p_leg    = [p_help[0], p2.get_lines()[-4], p2.get_lines()[-6], p2.get_lines()[-3], p2.get_lines()[-5], p2.get_lines()[-2]]
        txt_leg  = ['EURO-CORDEX models', 'EURO-CORDEX median', 'ERA5-Land', 'ERA5-Land median', 'E-OBS', 'E-OBS median']
        ncol     = 3
        EOBS_str = '_vEOBS'
    else:
        p_leg    = [p_help[0], p2.get_lines()[-3], p2.get_lines()[-4], p2.get_lines()[-2]]
        txt_leg  = ['EURO-CORDEX models', 'EURO-CORDEX median', 'ERA5-Land', 'ERA5-Land median']
        ncol     = 2
        EOBS_str = ''
        
    #Legend        
    ax.legend(p_leg, txt_leg, ncol=ncol, fontsize=14, loc=8, bbox_to_anchor=(-2.2, -1.4), frameon=False)
    
    #Save figure
    fig.savefig(dir_fig + 'FigS_Distribution_' + variab + '_1981-2010_JJA' + EOBS_str + '.png', dpi=200, bbox_inches='tight')
    

## Plot maps

In [ ]:
#Loop over variables
for varf, varn, varS, varo in zip(vars_ERA5[0:2], vars_ERA5[0:2], vars_STAT, var_names[0:2]):
    
    #Create figures
    fig, axes = plt.subplots(6, 6, figsize=(14, 10), subplot_kw=dict(projection=ccrs.PlateCarree()))        
    axes = axes.flatten()
    plt.subplots_adjust(hspace=0.25, wspace=0.1)

    #Loop over models
    for i1, city in enumerate(cities):

        #Get axes
        ax = axes[i1]
        ax.spines['geo'].set_edgecolor([0.8, 0.8, 0.8])

        #Get file name
        dir_variab = dir_ERA + varf + '/'
        fname = [file for file in os.listdir(dir_variab) if varf in file]
        if len(fname)==0:
            sys.exit('Filename not unique')                

        #Read data
        data = xr.open_dataset(dir_variab + fname[0])

        #Read station data
        fname_STA = dir_STA + varS + '_Stations_' + city + ".csv"
        if os.path.exists(fname_STA):
            data_STA = pd.read_csv(fname_STA)
            STA_exists = True
        else:
            STA_exists = False
        
        #Convert longitude from [0, 360] to [-180, 180]
        if 'longitude' in data.coords:  lat_name, lon_name = 'latitude', 'longitude'
        elif 'lon' in data.coords:      lat_name, lon_name = 'lat', 'lon'
        if data[lon_name].max()>180:
            data[lon_name] = data[lon_name].where(data[lon_name]<180, ((data[lon_name] + 180) % 360) - 180)

        #Get lat and lon of city
        lat_sel, lon_sel = city_coords[city]
        
        #Find grid point closest to city
        lat_city = np.argmin(np.abs(data[lat_name].values - lat_sel))
        lon_city = np.argmin(np.abs(data[lon_name].values - lon_sel))
        
        #Select NxN box around grid point
        N = 11
        N_half = int((N-1)/2)
        lat_rng  = slice(lat_city - N_half, lat_city + N_half + 1)
        lon_rng  = slice(lon_city - N_half, lon_city + N_half + 1)
        data_sel = data.isel(latitude=lat_rng, longitude=lon_rng)        

        #Convert K to °C
        data_plot = data_sel - 273.15
        
        #Define min and max for colorbar
        d1 = np.nanquantile(data_plot[varf].values, 0.05)
        d2 = np.nanquantile(data_STA.iloc[:,2], 0.05)
        vmin = np.min([d1, d2])
        
        d1 = np.nanquantile(data_plot[varf].values, 0.95)
        d2 = np.nanquantile(data_STA.iloc[:,2], 0.95)
        vmax = np.max([d1, d2])
        
        #Adjust coordinates
        try:
            LON, LAT = mpu.infer_interval_breaks(data_plot[lon_name], data_plot[lat_name])
        except:
            LON, LAT = data_plot[lon_name], data_plot[lat_name]

        #Add coastlines and borders
        ax.coastlines(resolution='50m', linewidth=1, color='#444444', zorder=5)                
            
        #Plot ERA5
        hp = ax.pcolormesh(LON, LAT, data_plot[varn], vmin=vmin, vmax=vmax, cmap='RdBu_r', zorder=3)
        
        #Plot stations
        if STA_exists:
            ax.scatter(data_STA.iloc[:,1], data_STA.iloc[:,0], c=data_STA.iloc[:,2], transform=ccrs.PlateCarree(),
                       edgecolor='k', vmin=vmin, vmax=vmax, cmap='RdBu_r', zorder=12)
        
        #Plot city center
        ax.scatter(lon_sel, lat_sel, 50, transform=ccrs.PlateCarree(), color='k', marker='x', linewidth=2, zorder=10)

        if city=='NizhnyNovgorod':     city_out = 'Nizhny Novgorod'
        elif city=='SaintPetersburg':  city_out = 'Saint Petersburg'
        else:                          city_out = city
            
        #Set extent and title
        ax.set_extent([lon_sel - 0.6, lon_sel + 0.6, lat_sel - 0.5, lat_sel + 0.5])
        ax.set_title(city_out)

        #Colorbar
        cbar = mpu.colorbar(hp, ax, orientation='vertical', extend='both', size=0.05, shrink=0.1, pad=0.05)
        cbar.ax.tick_params(labelsize=10)
        if np.mod(i1,6)==5:
            cbar.set_label(varS + ' [°C]', fontsize=11)    
    
#     #Save figure
#     fig.savefig(dir_fig + 'FigS_UHI_ERA5_stations_' + varo + '_' + str(N) + 'x' + str(N) + '_1981-2005_JJA.png', dpi=200, bbox_inches='tight')
    